In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install igraph


In [ ]:
import numpy as np
import pandas as pd 
import os 
import random
import math
from statistics import mean
import matplotlib.pyplot as plt 
from scipy.stats import ttest_ind
import more_itertools
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
from datetime import datetime
from sklearn.cluster import KMeans
#import igraph as ig
import json
import scipy.io
import statistics
from tqdm import tqdm

In [ ]:
#path pour Mathusan
PATH_DATA = '/content/drive/Othercomputers/My Laptop (1)/Systeme de Recommendation/TP/TP2/Data'
#PATH_DATA = '/content/drive/Othercomputers/My Laptop/TP2/'

#PATH_DATA = "/content/drive/MyDrive/Colab Notebooks/POLTECHNIQUE/Hiver2022/System de recommendation/TP2"
#PATH_DATA='/content/drive/MyDrive/POLYTECHNIQUE/Hiver2022/System de recommendation/TP2'

In [ ]:
# # items 
# test_data = pd.read_csv(os.path.join(PATH_DATA, '12-articles.csv'))
# # # users
# with open(os.path.join(PATH_DATA, 'dblpv13.json')) as file:
#   train_data=json.load(file)
# # # votes
# # votes = json.load(os.path.join(PATH_DATA, 'votes.csv'), sep='|')

In [ ]:
#read adjacency matrix file
adjacency_matrix = scipy.io.mmread(os.path.join(PATH_DATA, 'tp-matrix.dgt'))
adjacency_matrix=adjacency_matrix.tocsr()
#treating entries more than 1
adjacency_matrix[adjacency_matrix>1]=1
#reading articles names file
node_names=pd.read_csv(os.path.join(PATH_DATA, 'tp-matrix-names.csv'), sep="\s+")
node_names.rename(columns={'x':'titre'},inplace=True)
#reading test file
test_nodes=pd.read_csv(os.path.join(PATH_DATA, '12-articles.csv'))
# node_names=node_names.reset_index()

#Q.1

In [ ]:
#calculate pageRank the maximum iteration is 1000 but the if the difference between current step and previous step is less than epsilon the function will stop
def calcul_general_page_rank(adjacency_matrix,d=0.85, max_iter=1000, epsilon=1e-5):
  #initialize page rank
  r_prev=np.array([[1]]*adjacency_matrix.shape[0])
  for i in range(max_iter):
    #to avoid division by 0, the denominator  increases by 1
    r_cur=np.array((1-d)/(adjacency_matrix.shape[0])+d*(adjacency_matrix.T*(r_prev/(adjacency_matrix.sum(axis=1)+1))))
    if np.mean(np.abs(r_cur-r_prev))<1e-5:
      #to know the number of iteration to achive convergence we print i
      #print(f'number of iteration was: {i}')
      break
    else:
      r_prev=r_cur

  return r_cur


In [ ]:
r=calcul_general_page_rank(adjacency_matrix)
#using transpose and the first element of r give a row vector of (50497,)
r.T[0]

array([6.57674773e-06, 5.36145139e-04, 2.39036472e-05, ...,
       2.97047349e-06, 3.47545399e-06, 2.97047349e-06])

In [ ]:
def calc_mrr(test_nodes):
  #list of mean of mrr of test articles
  mrr_test=[]
  for id in test_nodes['id']:
    #relevant articles based on adjacency matrix rows
    pertinent_articles_index=adjacency_matrix[node_names[node_names["titre"]==id].index-1].nonzero()[1]

    # pagerank(r)
    r=calcul_general_page_rank(adjacency_matrix)
    #sorting index of relevant article using page rank in ascending order and getting top 10
    top_articles=np.argsort(r.T[0][pertinent_articles_index])[::-1]

    #getting title of top 10 article it is commented


    #calculate MRR

    #sorting index pagerank list
    r_index_sorted=np.argsort(r.T[0])[::-1]
    #list of mrr for recommendations of an article 
    mrr_list=[]

    for article in pertinent_articles_index[list(top_articles)]:
      #find the rank of each recommended article
      rank_relevant_article=np.where(r_index_sorted==article)
      #appending mrr of each artcle to the list, add 1 to index to consider the difference between rank and index
      mrr_list.append(1/(rank_relevant_article[0][0]+1))
    
    #mean of mrr of recommended articles for an article

    mrr_mean=statistics.mean(mrr_list)
    #appending the mean of mrr for each test artcle
    mrr_test.append(mrr_mean)
  
    print(f' MRR for article id:{id} is: {mrr_mean}')
  meant_total=statistics.mean(mrr_test)
  print(f'\n ***Mean MRR for 12 test article is:*** {meant_total}')
  #return mrr_test, meant_total

calc_mrr(test_nodes)

  

 MRR for article id:53e9aa4fb7602d97033bee00 is: 0.00035932416665122093
 MRR for article id:53e9b098b7602d9703b03c4e is: 0.00042097762611217903
 MRR for article id:53e9b708b7602d97042a150f is: 6.77031438904695e-05
 MRR for article id:53e9ba6ab7602d970468b7d5 is: 0.0003019991384712623
 MRR for article id:53e9ad6cb7602d9703755d9b is: 0.01233074966658952
 MRR for article id:558b5658e4b037c0875c5887 is: 0.0009192031308223445
 MRR for article id:558b416a84ae84d265c25bff is: 0.004041173835406845
 MRR for article id:53e99984b7602d97021c6204 is: 0.0018726131581843083
 MRR for article id:53e9b917b7602d97044fd88f is: 0.001547271902455532
 MRR for article id:53e9b360b7602d9703e3f04a is: 0.000854077693163559
 MRR for article id:53e9b879b7602d9704446291 is: 0.0006752420777899296
 MRR for article id:53e99bc0b7602d970246a745 is: 0.00013024084239606583

 ***Mean MRR for 12 test article is:*** 0.00196004803182777


## Analysis  
In this part we calculated general pagerank. It means that we give the same probability of random walk (teleportation) to all articles in the set. In the nex part we will try to be more subject oriented and limit teleportation to articles that are relevant and similar to the target article.

# Question 2

In [ ]:
#FINAL VERSION OF THE FINAL VERSION OF THE FINAL VERSION exposant final version !!!

In [ ]:
#new version
#calculating the resulting sparse vector from the summation of different levels of relation between the ajdacency matrix
def calc_order(adjacency_matrix, ind):
  ordre_1=adjacency_matrix[ind]
  ordre_n1=adjacency_matrix.T[ind]
  ordre_11=ordre_1.dot(adjacency_matrix)
  ordre_1n1=ordre_1.dot(adjacency_matrix.T)
  ordre_n11=ordre_n1.dot(adjacency_matrix)
  ordre_1n11=ordre_1.dot(adjacency_matrix.T.dot(adjacency_matrix))
  ordre_n111=ordre_n1.dot(adjacency_matrix.dot(adjacency_matrix))
  order=ordre_1+ordre_n1+ordre_11+ordre_1n1+ordre_1n11+ordre_1n11+ordre_n111
  # order=adjacency_matrix+adjacency_matrix.T+adjacency_matrix.multiply(adjacency_matrix)+adjacency_matrix.multiply(adjacency_matrix.T)+adjacency_matrix.T.multiply(adjacency_matrix)+adjacency_matrix.multiply(adjacency_matrix.T)*adjacency_matrix.T+adjacency_matrix.T*adjacency_matrix*adjacency_matrix
  #making sure the resulting vector is binary
  order[order>1]=1
  return order

In [ ]:
#new version
def calcul_thematique_rank2(adjacency_matrix,thematic_matrix, d=0.85,  max_iter=1000, epsilon=1e-5):
  #initialize page rank
  dict={}
  # for i in tqdm(range(adjacency_matrix.shape[0])):
  r_prev=np.array([[0]]*adjacency_matrix.shape[0])
  #thematic_matrix=calc_order(adjacency_matrix) #calculate thematic matrix
  thematic_articles_index=thematic_matrix.nonzero()[1]
  #initialize the pagerank to the values of the thematic vector
  r_prev[thematic_articles_index]=1
  teleport=r_prev.copy()
  #iterate the pagerank so that the difference between the previous and the current pagerank is quite the same
  for i in range(max_iter):
    #calculating the pagerank 
    r_cur=np.array(((1-d)/len(teleport[teleport==1])*teleport)+d*(adjacency_matrix.T*(r_prev/(adjacency_matrix.sum(axis=1)+1)))) #(m_l_o.T*(r_prev/(m_l_o.sum(axis=1)+1))))
    if np.mean(np.abs(r_cur-r_prev))<1e-5:
      #to know the number of iteration to achive convergence we print i

      break
    else:
      #updating the current and previous pagerank
      r_prev=r_cur

  return r_cur
#id=test_nodes['id'][9]#3000 (,3262,3276)
#articles=adjacency_matrix[node_names[node_names["titre"]==id].index-1]
#articles_sparse_original=articles.toarray()[0]
#calcul_general_page_rank2(adjacency_matrix,articles_sparse_original)

In [ ]:
#main function to calculate the mrr through the use of the methods above

def mrr_q2(test_nodes,adjacency_matrix):    
    results=[]
    for id in test_nodes['id']:#3000 (,3262,3276)
        #get the index of the current test id in the adjacency matrix
        ind=node_names[node_names["titre"]==id].index[0]-1
        #print the title of the test if 
        print('**',test_nodes.loc[test_nodes['id']==id,'titre'].iloc[0],'**')
        #get the line for the given title
        articles_ref=adjacency_matrix[node_names[node_names["titre"]==id].index-1]
        #calculate the resulting  (transformed matrix)
        article_them=calc_order(adjacency_matrix,ind)

        #transfer the vector of the given index from the original matrix
        articles_sparse_original_ref=articles_ref.toarray()[0]
        #transfer the vector of the given index from the transformed matrix
        articles_sparse_original_them=article_them.toarray()[0]

        #calculate the ranking for the nul values by calculating the median of their ranks (as instructed by the statement)
        rang_nulle=sum(list(range(np.count_nonzero(articles_sparse_original_them),len(articles_sparse_original_them))))/(len(articles_sparse_original_them)-np.count_nonzero(articles_sparse_original_them))
        rank_lov=[]
        #iterate over the non zero articles of the vector
        for a in list(np.nonzero(articles_sparse_original_ref)[0]):
            #turn the value of the non zero (a) to zero
            article_sparse=articles_sparse_original_ref.copy()
            article_sparse[a]=0
            adjacency_thematique=adjacency_matrix.copy()
            adjacency_thematique[ind,a]=0
            #perform the thematic adjacency over the modified a->0 vector
            thematique=calc_order(adjacency_thematique,ind)
            
            #calculate the rank from the resulting vector of the thematic + a->0 matrix
            rang_sort=calcul_thematique_rank2(adjacency_matrix,thematique)
            #sort the ranks
            rang_sort=np.argsort(rang_sort.T[0])[::-1]
            #initialize a mrr list
            mrr_list=[]
            #fill the rank elif the thematic value for the coordinates are still 0 or not
            if thematique[0,a]==0:
                mrr_list.append(1/rang_nulle)
            else:

                rang=np.where(rang_sort==a)[0][0]+1
                mrr_list.append(1/rang)
            #fill the values for the other coordinates of the vector which is not equal to a
            for b in list(np.nonzero(articles_sparse_original_ref)[0]):
            #for b in list(np.nonzero(article_sparse)):
                if b!=a:
                    rang_rest=np.where(rang_sort==b)[0][0]+1
                    mrr_list.append(1/rang_rest)
            #fill the rank_lov array with the mean of the mrr
            rank_lov.append(statistics.mean(mrr_list))
        print("For which the MRR is :")
        results.append(statistics.mean(rank_lov))
        print(statistics.mean(rank_lov))
    print(f'\n ***Mean MRR for 12 test article is:*** {statistics.mean(results)}')
        

mrr_q2(test_nodes,adjacency_matrix)

** Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks **
For which the MRR is :
0.01089403395719747
** Qualitative organization of collections of shapes via quartet analysis **
For which the MRR is :
0.005754574109713856
** Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management **
For which the MRR is :
0.10151231295384355
** Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires **
For which the MRR is :
0.30857986861758036
** Randomized locality sensitive vocabularies for bag-of-features model **
For which the MRR is :
0.06807897673591592
** Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization **
For which the MRR is :
0.016069834081623147
** Discrete tracking of parametrized curves **
For which the MRR is :
0.08919002869064693
** Spatial template extraction for image retrieval by region matching. **
For which the MRR is :
0.040278690888093376
*

## Analysis 
Performing a PageRank on all the articles but keeping only the relevant articles for the calculation of the 12 components of the MRR, we get a much better MRR (0.0638595545673379) than in Question 1. In this part we increase the probability of random walk to articles that only are present in the Thematic graph. Therefore their PR increases comparing to other articles and they will have higher rank and MRR will increases. Please note that the probability of random walk to non-relevant articles are 0 here.



# Qestion 3

In [ ]:
def article_recom_similarity(adjacency_matrix,node_names,test_nodes,Thematic=False):
  #initialize a dictionnary of similarity and a mrr array
  dict_similar={}
  mrr_total=[]
  #thematic_matrix=calc_order(adjacency_matrix)
  #iterate over the ids of the test_node variable df
  for id in test_nodes['id']:

    #article index adjusted for adjacency matrix index
    article_idx=node_names[node_names["titre"]==id].index-1
    #find the values that are non zero in the vector
    pertinent_articles_index=adjacency_matrix[article_idx].nonzero()[1]

    #create a dictionary of the frequency of recommendation for each article
    dict_similar[id]={}
    mrr_test=[]
    print('**',test_nodes.loc[test_nodes['id']==id,'titre'].iloc[0],'**')
    #iterate over the pertinent article index which are non zero
    for a in pertinent_articles_index:
      #copy the adjacency matrix
      M_ad=adjacency_matrix.copy()
      #leave one out by equalling the coordinate to zero
      M_ad[article_idx,a]=0
      #perform the thematic calculation of the vector (similar to #2)
      if Thematic:
        them_vector=calc_order(M_ad,article_idx)
        #find the cosine similarity for the result of the thematic vector
        sim=np.array([cosine_similarity(them_vector, M_ad)[0]])
        
      else:
        #perform the cosine similarity over the regular adjacency matrix
        sim=np.array([cosine_similarity(M_ad[article_idx], M_ad)[0]])
        
      sim[sim>0]=1
      sim[0,article_idx]=0
      
      r=calcul_thematique_rank2(M_ad,sim)
      rang_nulle=sum(list(range(np.count_nonzero(sim),len(sim))))/(len(sim)-np.count_nonzero(sim))

      #put cosine similarity of the article with itself to 0
      #sim[article_idx]=0
      #calculate pagerank
      r_index_sorted=np.argsort(r.T[0])[::-1]

      #sort and find the length of the non zero variable to keep them
      similar_article=np.argsort(sim)[::-1][:len(sim[sim>0])][0]
      #initialize an empty mrr array
      mrr_list=[]
      if a in similar_article:
        #print("Yes")
        rank_relevant_article=np.where(r_index_sorted==a)[0][0]+1
        #appending mrr of each artcle to the list
        mrr_list.append(1/rank_relevant_article)
      else:
        #print("NO")
        mrr_list.append(1/rang_nulle)
      
      for b in pertinent_articles_index:
        if b!=a:
          rang_rest=np.where(r_index_sorted==b)[0][0]+1
          mrr_list.append(1/rang_rest)
      mrr_test.append(statistics.mean(mrr_list))
    print("For which the MRR is :")
    print(statistics.mean(mrr_test))
    mrr_total.append(statistics.mean(mrr_test))
  
  meant_total=statistics.mean(mrr_total)
  print(f'\n ***Mean MRR for 12 test article is:*** {meant_total}')
  
  #we can also print the top recommended articles based on similarity if necessary 
      
  # print(dict_similar)
  # for id in dict_similar.keys():
  #   mrr_list=[]
  #   print(id)
  #   print("using Leave one out cross validation:")
  #   print the articles as recommendation system that have been recommended more than others during Leave-One-Out
  #   recom_list=list(sorted(dict_similar[id], key=dict_similar[id].get, reverse=True))[:10] 
  #   print(recom_list)
    



In [ ]:
article_recom_similarity(adjacency_matrix,node_names,test_nodes)

** Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks **
For which the MRR is :
0.17215400196174838
** Qualitative organization of collections of shapes via quartet analysis **
For which the MRR is :
0.11558261988955752
** Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management **
For which the MRR is :
0.07766800803288629
** Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires **
For which the MRR is :
0.4418463441500984
** Randomized locality sensitive vocabularies for bag-of-features model **
For which the MRR is :
0.08839229795927607
** Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization **
For which the MRR is :
0.05891633562552609
** Discrete tracking of parametrized curves **
For which the MRR is :
0.16089212248452056
** Spatial template extraction for image retrieval by region matching. **
For which the MRR is :
0.1453262127223392
** Dis

In [ ]:
article_recom_similarity(adjacency_matrix,node_names,test_nodes,Thematic=True)

** Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks **
For which the MRR is :
0.001501486834851921
** Qualitative organization of collections of shapes via quartet analysis **
For which the MRR is :
0.0006459870156159253
** Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management **
For which the MRR is :
0.0013891903462509435
** Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires **
For which the MRR is :
0.0005702031457593413
** Randomized locality sensitive vocabularies for bag-of-features model **
For which the MRR is :
0.01690266448343102
** Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization **
For which the MRR is :
0.0020666930319984925
** Discrete tracking of parametrized curves **
For which the MRR is :
0.0075574706503116346
** Spatial template extraction for image retrieval by region matching. **
For which the MRR is :
0.00231300877

##Analysis  
In Q3 first we calculate similarity of each test article with other articles in the adjacency matrix. We used two methods to consider test article as vector space: 1- references of the article, 2- Thematic graph like Q2. As the result shows considering references gives better MRR because in this case the teleportation set is small and therefore the probability of teleportation increases and since the references is more probably in the similar articles, we have higher rank for them.  
  

But in the case of similarity for thematic matrix the set is big, and it is quite close to adjacency matrix, so the teleportation probability is quite close to random walk in general pagerank ((1-d)/N).  
  

In conclusion we can say as we move toward more topic specific (personalized) set in teleportation and limit teleportation set to the articles that we want we will get better MRR. 


#Question 4

In [ ]:
#ranking the personalized pageranks with the similarity method
def calcul_rank_personalise(adjacency_matrix,thematic_matrix,article_idx, d=0.85,  max_iter=1000, epsilon=1e-5):
  #initialize page rank

  #initialize this time the initial pagerank to 1s
  r_prev=np.array([[1]]*adjacency_matrix.shape[0])

  
  #create the undirected graph
  non_directed_mat=adjacency_matrix.T+adjacency_matrix
  non_directed_mat[non_directed_mat>1]=1
  #calculate the similarity between the thematic and undirected graph
  sim=np.array(cosine_similarity(thematic_matrix, non_directed_mat))

  #putting our article index from the similarity to 0
  sim[0,article_idx]=0

  #we performed a hyper-parameter research for this part that will be explained in the analysis
  # this variable contains the best 99.9% of the similarities indexes for the given thematic_matrix
  similar_articles_index=np.where(sim>np.quantile(sim,0.999))[1]

  #the teleportation matrix is the one that will let the random jump to be focused towards articles with higher similarity

  #fix the teleportation matrix to the similarity+threshold
  teleport=np.array([[0]]*adjacency_matrix.shape[0])
  teleport[similar_articles_index]=1

  #iterate until convergence of the previous-current pagerank
  for i in range(max_iter):
    #calculating the current pagerank
    r_cur=np.array(((1-d)/len(teleport[teleport==1])*teleport)+d*(adjacency_matrix.T*(r_prev/(adjacency_matrix.sum(axis=1)+1)))) #(m_l_o.T*(r_prev/(m_l_o.sum(axis=1)+1))))
    if np.mean(np.abs(r_cur-r_prev))<1e-5:
      #to know the number of iteration to achive convergence we print i
      #print(f'number of iteration was: {i}')
      break
    else:
      #convergence process
      r_prev=r_cur

  return r_cur



In [ ]:
#main function
def PPR(test_nodes,adjacency_matrix):    
    results=[]
    #iterate over the ids of test_nodes
    for id in test_nodes['id']:#3000 (,3262,3276)
        #keep the index of the id
        ind=node_names[node_names["titre"]==id].index[0]-1
        print('**',test_nodes.loc[test_nodes['id']==id,'titre'].iloc[0],'**')

        #find the row for the given id on the original adjacency matrix
        articles_ref=adjacency_matrix[node_names[node_names["titre"]==id].index-1]
        # finding it on the thematic matrix
        article_them=calc_order(adjacency_matrix,ind)

        #keeping the original line for the id
        articles_sparse_original_ref=articles_ref.toarray()[0]
        #and the thematic one also
        articles_sparse_original_them=article_them.toarray()[0]

        rank_lov=[]
        #iterate over the non zero values of the original line/vector
        for a in list(np.nonzero(articles_sparse_original_ref)[0]):
          
            #leave-one out method by turning the a value to 0 on the adjacency matrix
            article_sparse=articles_sparse_original_ref.copy()
            article_sparse[a]=0
            adjacency_thematique=adjacency_matrix.copy()
            adjacency_thematique[ind,a]=0

            #calculating the thematic vector for the index of ind from the new adjacency matrix
            #thematique=calc_order(adjacency_thematique,ind)
            thematique=adjacency_thematique[ind]


            #ranking through the personnalized pagerank
            rang_sort=calcul_rank_personalise(adjacency_matrix,thematique,ind)
            #sorting
            rang_sort=np.argsort(rang_sort.T[0])[::-1]
            mrr_list=[]

            #calculating the mrr for the non zero values of the vector
            for b in list(np.nonzero(articles_sparse_original_ref)[0]):

              rang=np.where(rang_sort==b)[0][0]+1
              mrr_list.append(1/rang)
            rank_lov.append(statistics.mean(mrr_list))
        print("For which the MRR is :")
        results.append(statistics.mean(rank_lov))
        print(statistics.mean(rank_lov))
    print(f'\n ***Mean MRR for 12 test article is:*** {statistics.mean(results)}')


PPR(test_nodes,adjacency_matrix)

** Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks **
For which the MRR is :
0.18998268520905115
** Qualitative organization of collections of shapes via quartet analysis **
For which the MRR is :
0.12961742202601714
** Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management **
For which the MRR is :
0.0435160126299041
** Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires **
For which the MRR is :
0.4069865996308824
** Randomized locality sensitive vocabularies for bag-of-features model **
For which the MRR is :
0.1613555341245791
** Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization **
For which the MRR is :
0.06609447524725721
** Discrete tracking of parametrized curves **
For which the MRR is :
0.1476563645228442
** Spatial template extraction for image retrieval by region matching. **
For which the MRR is :
0.1525527776817748
** Distan

## Analysis
We have proceeded to do a mixture of question 2 and question 3 to obtain a personnalized pagerank. Here, the difference from question 2 is the usage of the cosine similarity for every article in the test node between the articles index obtained because of the thematic modification of the article vector from the chain levels of relations. Another difference to question 2 is that we used the original content matrix contains all the articles because it was the one that provided the better results compared to the thematic matrix in Q3.

The difference from Question 3 is that we applied the similarity to the adjacency matrix for an undirected graph (relation 1 and relation -1 together) and we only took the best 99.9% similarities for the teleportation vector. This is basically a hyper-parameter that we decided to find by doing at first a greedy search, we tried different scalar values between 0.01 and 0.1. Following the error message that left us with the null vector for the teleportation matrix, we decided to try out the quantile method from the NumPy library. At first there was almost little to no improvement from our question 3 results for the quantiles ranging from 0 to 0.8. But soon after we also found out that if we demanded too little non null components in the teleportation vector, we would end with a very low MRR (try a quantile of 0.9999, for which mrr was 0.05). The ***optimal hyperparameter*** for us was therefore the 0.999 quantile for the teleportation matrix for every single non null article in the adjacency vector given a test node. This article-by-article similarity cutting points (quantile) is the part makes our method very personalized. 

In fact this is also reflected by a better resulf of mrr than any method we have tried before : 0.14503476499807869.

There are other methods that could be better in terms of personnalization  (like using the Citation Network Dataset V13 in its entirety but we would have calculated a similary through the 'keyword' or a tokenized 'indexed_abstract' variable. But these methods would have cost us much more RAM and CPU usage. And unfortunately we are not equipped for these much complexe on such dense graphs. We may have a very decent and optimal-complexity method for the personnalized pagerank considering our limited ressources. 


